# Introduction

In this tutorial we will deploy a simplest possible model to Cognite Model Hosting from a notebook. A model that convert from fahrenheit to celcius. Of course, having a model for such a simple task would usually be overkill, but it's a nice example. Deploying a notebook is great for simple models and trying out things. For more complex models with a lot of code and many dependencies you should consider creating a proper source package.

**Requirements**:
- You need to have the `cognite-model-hosting-notebook` package installed
- The environment variable `COGNITE_API_KEY` should be set

# Defining our model

First off we need to define our dependencies. This is simply achieved by creating a code cell that start with `# !requirements`. Since we have no dependencies we'll leave the rest of the cell empty.

In [1]:
# !requirements

Then we need to specify the code the prediction routine of our model will run.

In [2]:
# !model
def predict(instance):
    return (instance - 32) * (5/9)

Notice that we start the cell with `# !model`. This way we say that this code should be included in our model. The next cells below will only be for local execution, and are not supposed to be ran in Model Hosting. And thus do not start with `# !model`.

Okey, now we have defined our model. Let's try it out locally before we deploy it to Model Hosting.

In [3]:
predict(70)

21.11111111111111

Seems to be working nicely!

# Deploying our model

It's time to deploy our model. Let's first import what we need.

In [4]:
from cognite.client import CogniteClient
from cognite.model_hosting.notebook import deploy_model_version

model_hosting = CogniteClient().experimental.model_hosting

To deploy a model version we first need a model.

In [5]:
model_id = model_hosting.models.create_model("fahrenheit-to-celsius").id

And then we deploy this notebook as a model version. It's important you **save the notebook** before doing this since the notebook file will be read to find your model.

In [6]:
model_version_id = deploy_model_version(
    name="fahrenheit-celsius-v1",
    model_id=model_id,
    runtime_version="0.1"
)

Now we just have to wait for the deployment to finish. This usually takes a few minutes. Notice that the next step won't work before the status of the model version is 'READY'.

In [7]:
model_hosting.models.get_model_version(model_id, model_version_id).status

'READY'

# Testing the model

Now that's it deployed we can test it out.

In [8]:
model_hosting.models.online_predict(model_id, model_version_id, instances=[70, 80, 90])

[21.11111111111111, 26.666666666666668, 32.22222222222222]

It's important to note that these predictions were performed in Cognite Model Hosting in the cloud, not on your computer. Anyone with internet (and appropriate access rights) can now access this model!

When you're done - remember to clean up after yourself:

In [9]:
source_package_id = model_hosting.models.get_model_version(model_id, model_version_id).source_package_id
model_hosting.models.delete_model(model_id)
model_hosting.source_packages.delete_source_package(source_package_id)